In [26]:
import h5py
import numpy as np
import pandas as pd
import folium
import rasterio
from rasterio.transform import from_bounds
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import os
from pathlib import Path

# 設定資料路徑
data_dir = Path("/Users/insight_ai/我的雲端硬碟/share/防災materials/nasa/VNP46A4_1-20251004_081026")
hdf_files = list(data_dir.glob("*.h5"))

print(f"找到 {len(hdf_files)} 個 HDF5 檔案:")
for file in hdf_files:
    print(f"  - {file.name}")


找到 4 個 HDF5 檔案:
  - VNP46A4.A2023001.h30v06.001.2024019015120.h5
  - VNP46A4.A2021001.h30v06.001.2022094124511.h5
  - VNP46A4.A2022001.h30v06.001.2023082112411.h5
  - VNP46A4.A2020001.h30v06.001.2021125121127.h5


In [36]:
import h5py
import numpy as np
import rasterio
from rasterio.transform import from_bounds
from rasterio.crs import CRS
from pathlib import Path

# 設定資料路徑
data_dir = Path("/Users/insight_ai/我的雲端硬碟/share/防災materials/nasa/VNP46A4_1-20251004_081026")
hdf_files = list(data_dir.glob("*.h5"))

print(f"找到 {len(hdf_files)} 個 HDF5 檔案:")
for file in hdf_files:
    print(f"  - {file.name}")

# 處理每個 HDF5 檔案
for idx, hdf_file in enumerate(hdf_files):
    print(f"\n處理檔案 {idx+1}/{len(hdf_files)}: {hdf_file.name}")
    
    with h5py.File(hdf_file, 'r') as f:
        # 萃取資料
        data_path = 'HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/AllAngle_Composite_Snow_Free'
        snow_free_data = f[data_path][:]
        
        # 萃取座標資訊
        lat_path = 'HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/lat'
        lon_path = 'HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/lon'
        latitude = f[lat_path][:]
        longitude = f[lon_path][:]
        
        # 取得資料屬性
        attrs = dict(f[data_path].attrs)
        
        # 處理填值
        fill_value = attrs['_FillValue'][0]
        scale_factor = attrs['scale_factor'][0]
        offset = attrs['offset'][0]
        
        # 轉換為實際數值（使用 float32 保留原始數值範圍）
        valid_mask = snow_free_data != fill_value
        actual_data = np.where(valid_mask, 
                              snow_free_data * scale_factor + offset, 
                              np.nan).astype(np.float32)  # 明確指定 float32
        
        print(f"  資料形狀: {snow_free_data.shape}")
        print(f"  資料類型: {actual_data.dtype}")
        print(f"  緯度形狀: {latitude.shape}, 範圍: {latitude.min():.4f} 到 {latitude.max():.4f}")
        print(f"  經度形狀: {longitude.shape}, 範圍: {longitude.min():.4f} 到 {longitude.max():.4f}")
        print(f"  有效資料點數: {np.sum(valid_mask)}")
        print(f"  實際數值範圍: {np.nanmin(actual_data):.6f} 到 {np.nanmax(actual_data):.6f}")
        
        # 建立 GeoTIFF
        # 檢查座標維度並建立轉換矩陣
        if latitude.ndim == 1 and longitude.ndim == 1:
            # 1D 座標陣列
            lat_min, lat_max = latitude.min(), latitude.max()
            lon_min, lon_max = longitude.min(), longitude.max()
        elif latitude.ndim == 2:
            # 2D 座標陣列
            lat_min, lat_max = latitude.min(), latitude.max()
            lon_min, lon_max = longitude.min(), longitude.max()
        else:
            print(f"  警告: 未預期的座標維度")
            continue
        
        # 建立 affine 轉換
        rows, cols = actual_data.shape
        transform = from_bounds(lon_min, lat_min, lon_max, lat_max, cols, rows)
        
        # 輸出檔名
        output_file = data_dir / f"{hdf_file.stem}_snow_free_float32.tif"
        
        # 寫入 GeoTIFF - 關鍵：使用 float32 類型
        with rasterio.open(
            output_file,
            'w',
            driver='GTiff',
            height=rows,
            width=cols,
            count=1,
            dtype=rasterio.float32,  # 明確使用 float32
            crs=CRS.from_epsg(4326),  # WGS84
            transform=transform,
            nodata=np.nan,
            compress='lzw',
            tiled=True,  # 使用 tile 結構提升效能
            blockxsize=256,
            blockysize=256
        ) as dst:
            dst.write(actual_data, 1)
            
            # 寫入元資料
            dst.update_tags(
                units=attrs.get('units', b'unknown').decode() if isinstance(attrs.get('units'), bytes) else attrs.get('units', 'unknown'),
                scale_factor=str(scale_factor),
                offset=str(offset),
                fill_value=str(fill_value),
                source_file=hdf_file.name,
                description='VIIRS DNB AllAngle Composite Snow Free - Original float32 values'
            )
        
        print(f"  ✓ 已儲存至: {output_file.name}")

print(f"\n完成! 共處理 {len(hdf_files)} 個檔案")

# 驗證數值範圍
print("\n=== 驗證 GeoTIFF 數值範圍 ===")
tif_files = list(data_dir.glob("*_snow_free_float32.tif"))

for tif_file in tif_files:
    with rasterio.open(tif_file) as src:
        data = src.read(1)
        print(f"\n{tif_file.name}:")
        print(f"  資料類型: {data.dtype}")
        print(f"  數值範圍: {np.nanmin(data):.6f} 到 {np.nanmax(data):.6f}")
        print(f"  NoData 值: {src.nodata}")
        print(f"  元資料: {src.tags()}")

找到 4 個 HDF5 檔案:
  - VNP46A4.A2023001.h30v06.001.2024019015120.h5
  - VNP46A4.A2021001.h30v06.001.2022094124511.h5
  - VNP46A4.A2022001.h30v06.001.2023082112411.h5
  - VNP46A4.A2020001.h30v06.001.2021125121127.h5

處理檔案 1/4: VNP46A4.A2023001.h30v06.001.2024019015120.h5
  資料形狀: (2400, 2400)
  資料類型: float32
  緯度形狀: (2400,), 範圍: 20.0042 到 30.0000
  經度形狀: (2400,), 範圍: 120.0000 到 129.9958
  有效資料點數: 479351
  實際數值範圍: 0.000000 到 573.599976
  ✓ 已儲存至: VNP46A4.A2023001.h30v06.001.2024019015120_snow_free_float32.tif

處理檔案 2/4: VNP46A4.A2021001.h30v06.001.2022094124511.h5
  資料形狀: (2400, 2400)
  資料類型: float32
  緯度形狀: (2400,), 範圍: 20.0042 到 30.0000
  經度形狀: (2400,), 範圍: 120.0000 到 129.9958
  有效資料點數: 479502
  實際數值範圍: 0.000000 到 285.500000
  ✓ 已儲存至: VNP46A4.A2021001.h30v06.001.2022094124511_snow_free_float32.tif

處理檔案 3/4: VNP46A4.A2022001.h30v06.001.2023082112411.h5
  資料形狀: (2400, 2400)
  資料類型: float32
  緯度形狀: (2400,), 範圍: 20.0042 到 30.0000
  經度形狀: (2400,), 範圍: 120.0000 到 129.9958
  有效資料點數: 479138
  實際數值

In [37]:
import h5py
import numpy as np
import rasterio
from rasterio.transform import from_bounds
from rasterio.crs import CRS
from pathlib import Path
import pandas as pd

# 設定資料路徑
data_dir = Path("/Users/insight_ai/我的雲端硬碟/share/防災materials/nasa/VNP46A4_1-20251004_081026")
hdf_files = list(data_dir.glob("*.h5"))

print(f"找到 {len(hdf_files)} 個 HDF5 檔案:")
for file in hdf_files:
    print(f"  - {file.name}")

# 儲存所有年度的資料
all_data = []
all_metadata = []

# 步驟 1: 讀取所有檔案的資料
print("\n=== 步驟 1: 讀取所有 HDF5 檔案 ===")
for idx, hdf_file in enumerate(hdf_files):
    print(f"\n處理檔案 {idx+1}/{len(hdf_files)}: {hdf_file.name}")
    
    with h5py.File(hdf_file, 'r') as f:
        # 萃取資料
        data_path = 'HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/AllAngle_Composite_Snow_Free'
        snow_free_data = f[data_path][:]
        
        # 萃取座標資訊
        lat_path = 'HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/lat'
        lon_path = 'HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/lon'
        latitude = f[lat_path][:]
        longitude = f[lon_path][:]
        
        # 取得資料屬性
        attrs = dict(f[data_path].attrs)
        
        # 處理填值
        fill_value = attrs['_FillValue'][0]
        scale_factor = attrs['scale_factor'][0]
        offset = attrs['offset'][0]
        
        # 轉換為實際數值
        valid_mask = snow_free_data != fill_value
        actual_data = np.where(valid_mask, 
                              snow_free_data * scale_factor + offset, 
                              np.nan).astype(np.float32)
        
        print(f"  資料形狀: {actual_data.shape}")
        print(f"  有效資料點數: {np.sum(~np.isnan(actual_data))}")
        print(f"  數值範圍: {np.nanmin(actual_data):.6f} 到 {np.nanmax(actual_data):.6f}")
        
        # 儲存資料
        all_data.append(actual_data)
        all_metadata.append({
            'filename': hdf_file.name,
            'latitude': latitude,
            'longitude': longitude,
            'shape': actual_data.shape,
            'attrs': attrs
        })

# 步驟 2: 檢查所有資料的空間範圍是否一致
print("\n=== 步驟 2: 檢查空間範圍一致性 ===")
shapes = [data.shape for data in all_data]
print(f"所有資料形狀: {shapes}")

if len(set(shapes)) == 1:
    print("✓ 所有檔案的空間網格一致，可以直接平均")
    same_grid = True
else:
    print("✗ 警告: 檔案的空間網格不一致")
    same_grid = False

# 取得座標資訊（使用第一個檔案的座標）
ref_metadata = all_metadata[0]
latitude = ref_metadata['latitude']
longitude = ref_metadata['longitude']

if latitude.ndim == 1 and longitude.ndim == 1:
    lat_min, lat_max = latitude.min(), latitude.max()
    lon_min, lon_max = longitude.min(), longitude.max()
elif latitude.ndim == 2:
    lat_min, lat_max = latitude.min(), latitude.max()
    lon_min, lon_max = longitude.min(), longitude.max()

print(f"座標範圍: 緯度 {lat_min:.4f} 到 {lat_max:.4f}, 經度 {lon_min:.4f} 到 {lon_max:.4f}")

# 步驟 3: 計算多年平均
print("\n=== 步驟 3: 計算多年平均 ===")

if same_grid:
    # 將所有資料堆疊成 3D 陣列 (年份, rows, cols)
    data_stack = np.stack(all_data, axis=0)
    print(f"資料堆疊形狀: {data_stack.shape}")
    
    # 計算平均值（忽略 NaN）
    mean_data = np.nanmean(data_stack, axis=0).astype(np.float32)
    
    # 計算標準差
    std_data = np.nanstd(data_stack, axis=0).astype(np.float32)
    
    # 計算有效年份數量（每個像素有多少個非 NaN 值）
    valid_count = np.sum(~np.isnan(data_stack), axis=0).astype(np.int16)
    
    print(f"平均值範圍: {np.nanmin(mean_data):.6f} 到 {np.nanmax(mean_data):.6f}")
    print(f"標準差範圍: {np.nanmin(std_data):.6f} 到 {np.nanmax(std_data):.6f}")
    print(f"有效年份數範圍: {np.nanmin(valid_count)} 到 {np.nanmax(valid_count)}")
    
    # 步驟 4: 儲存結果
    print("\n=== 步驟 4: 儲存結果 ===")
    
    rows, cols = mean_data.shape
    transform = from_bounds(lon_min, lat_min, lon_max, lat_max, cols, rows)
    
    # 儲存平均值
    output_mean = data_dir / "VIIRS_multi_year_mean.tif"
    with rasterio.open(
        output_mean,
        'w',
        driver='GTiff',
        height=rows,
        width=cols,
        count=1,
        dtype=rasterio.float32,
        crs=CRS.from_epsg(4326),
        transform=transform,
        nodata=np.nan,
        compress='lzw',
        tiled=True,
        blockxsize=256,
        blockysize=256
    ) as dst:
        dst.write(mean_data, 1)
        dst.update_tags(
            description=f'Multi-year mean of {len(all_data)} years',
            source_files=', '.join([m['filename'] for m in all_metadata]),
            units='nW/cm2/sr',
            statistics='mean'
        )
    print(f"✓ 平均值已儲存至: {output_mean.name}")
    
    # 儲存標準差
    output_std = data_dir / "VIIRS_multi_year_std.tif"
    with rasterio.open(
        output_std,
        'w',
        driver='GTiff',
        height=rows,
        width=cols,
        count=1,
        dtype=rasterio.float32,
        crs=CRS.from_epsg(4326),
        transform=transform,
        nodata=np.nan,
        compress='lzw',
        tiled=True,
        blockxsize=256,
        blockysize=256
    ) as dst:
        dst.write(std_data, 1)
        dst.update_tags(
            description=f'Multi-year standard deviation of {len(all_data)} years',
            source_files=', '.join([m['filename'] for m in all_metadata]),
            units='nW/cm2/sr',
            statistics='std'
        )
    print(f"✓ 標準差已儲存至: {output_std.name}")
    
    # 儲存有效年份數
    output_count = data_dir / "VIIRS_multi_year_count.tif"
    with rasterio.open(
        output_count,
        'w',
        driver='GTiff',
        height=rows,
        width=cols,
        count=1,
        dtype=rasterio.int16,
        crs=CRS.from_epsg(4326),
        transform=transform,
        nodata=-1,
        compress='lzw',
        tiled=True,
        blockxsize=256,
        blockysize=256
    ) as dst:
        dst.write(valid_count, 1)
        dst.update_tags(
            description='Number of valid years per pixel',
            source_files=', '.join([m['filename'] for m in all_metadata])
        )
    print(f"✓ 有效年份數已儲存至: {output_count.name}")
    
else:
    print("由於空間網格不一致，需要先進行重採樣對齊")
    print("請確認所有 HDF5 檔案是否來自相同的空間範圍")

# 步驟 5: 統計摘要
print("\n=== 步驟 5: 統計摘要 ===")
print(f"處理的檔案數: {len(all_data)}")
print(f"來源檔案:")
for m in all_metadata:
    print(f"  - {m['filename']}")

if same_grid:
    print(f"\n結果檔案:")
    print(f"  1. 多年平均: VIIRS_multi_year_mean.tif")
    print(f"  2. 標準差: VIIRS_multi_year_std.tif")
    print(f"  3. 有效年份數: VIIRS_multi_year_count.tif")

找到 4 個 HDF5 檔案:
  - VNP46A4.A2023001.h30v06.001.2024019015120.h5
  - VNP46A4.A2021001.h30v06.001.2022094124511.h5
  - VNP46A4.A2022001.h30v06.001.2023082112411.h5
  - VNP46A4.A2020001.h30v06.001.2021125121127.h5

=== 步驟 1: 讀取所有 HDF5 檔案 ===

處理檔案 1/4: VNP46A4.A2023001.h30v06.001.2024019015120.h5
  資料形狀: (2400, 2400)
  有效資料點數: 479351
  數值範圍: 0.000000 到 573.599976

處理檔案 2/4: VNP46A4.A2021001.h30v06.001.2022094124511.h5
  資料形狀: (2400, 2400)
  有效資料點數: 479502
  數值範圍: 0.000000 到 285.500000

處理檔案 3/4: VNP46A4.A2022001.h30v06.001.2023082112411.h5
  資料形狀: (2400, 2400)
  有效資料點數: 479138
  數值範圍: 0.000000 到 296.500000

處理檔案 4/4: VNP46A4.A2020001.h30v06.001.2021125121127.h5
  資料形狀: (2400, 2400)
  有效資料點數: 479380
  數值範圍: 0.000000 到 260.500000

=== 步驟 2: 檢查空間範圍一致性 ===
所有資料形狀: [(2400, 2400), (2400, 2400), (2400, 2400), (2400, 2400)]
✓ 所有檔案的空間網格一致，可以直接平均
座標範圍: 緯度 20.0042 到 30.0000, 經度 120.0000 到 129.9958

=== 步驟 3: 計算多年平均 ===
資料堆疊形狀: (4, 2400, 2400)


/var/folders/cs/s9sl30ms3yqc42bnr0xh29pr0000gn/T/ipykernel_22427/3772035123.py:100: RuntimeWarning: Mean of empty slice
  mean_data = np.nanmean(data_stack, axis=0).astype(np.float32)
/Users/insight_ai/我的雲端硬碟/share/防災materials/nasa/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


平均值範圍: 0.000000 到 259.274994
標準差範圍: 0.000000 到 265.731995
有效年份數範圍: 0 到 4

=== 步驟 4: 儲存結果 ===
✓ 平均值已儲存至: VIIRS_multi_year_mean.tif
✓ 標準差已儲存至: VIIRS_multi_year_std.tif
✓ 有效年份數已儲存至: VIIRS_multi_year_count.tif

=== 步驟 5: 統計摘要 ===
處理的檔案數: 4
來源檔案:
  - VNP46A4.A2023001.h30v06.001.2024019015120.h5
  - VNP46A4.A2021001.h30v06.001.2022094124511.h5
  - VNP46A4.A2022001.h30v06.001.2023082112411.h5
  - VNP46A4.A2020001.h30v06.001.2021125121127.h5

結果檔案:
  1. 多年平均: VIIRS_multi_year_mean.tif
  2. 標準差: VIIRS_multi_year_std.tif
  3. 有效年份數: VIIRS_multi_year_count.tif


In [38]:
import rasterio
import numpy as np
from pathlib import Path

data_dir = Path("/Users/insight_ai/我的雲端硬碟/share/防災materials/nasa/VNP46A4_1-20251004_081026")

# 驗證 GeoTIFF 檔案
print("=== 驗證 GeoTIFF 檔案 ===\n")

tif_file = data_dir / "VIIRS_multi_year_mean.tif"

with rasterio.open(tif_file) as src:
    # 讀取資料
    data = src.read(1)
    
    print(f"檔案: {tif_file.name}")
    print(f"資料類型: {data.dtype}")
    print(f"形狀: {data.shape}")
    print(f"CRS: {src.crs}")
    print(f"NoData: {src.nodata}")
    print(f"\n數值統計:")
    print(f"  最小值: {np.nanmin(data):.6f}")
    print(f"  最大值: {np.nanmax(data):.6f}")
    print(f"  平均值: {np.nanmean(data):.6f}")
    print(f"  中位數: {np.nanmedian(data):.6f}")
    print(f"  標準差: {np.nanstd(data):.6f}")
    print(f"\n百分位數:")
    print(f"  25%: {np.nanpercentile(data, 25):.6f}")
    print(f"  50%: {np.nanpercentile(data, 50):.6f}")
    print(f"  75%: {np.nanpercentile(data, 75):.6f}")
    print(f"  95%: {np.nanpercentile(data, 95):.6f}")
    print(f"  99%: {np.nanpercentile(data, 99):.6f}")
    
    print(f"\nNaN 值統計:")
    print(f"  總像素數: {data.size}")
    print(f"  NaN 數量: {np.sum(np.isnan(data))}")
    print(f"  有效數據: {np.sum(~np.isnan(data))}")
    
    # 檢查元資料
    print(f"\n元資料:")
    for key, value in src.tags().items():
        print(f"  {key}: {value}")
    
    # 檢查是否有異常值
    valid_data = data[~np.isnan(data)]
    print(f"\n有效資料範圍:")
    print(f"  最小值: {valid_data.min():.6f}")
    print(f"  最大值: {valid_data.max():.6f}")

# 同時檢查其他檔案
print("\n" + "="*50)
print("=== 檢查標準差檔案 ===\n")

std_file = data_dir / "VIIRS_multi_year_std.tif"
with rasterio.open(std_file) as src:
    std_data = src.read(1)
    print(f"標準差範圍: {np.nanmin(std_data):.6f} 到 {np.nanmax(std_data):.6f}")

print("\n" + "="*50)
print("=== 檢查有效年份數檔案 ===\n")

count_file = data_dir / "VIIRS_multi_year_count.tif"
with rasterio.open(count_file) as src:
    count_data = src.read(1)
    print(f"有效年份數:")
    unique, counts = np.unique(count_data[count_data >= 0], return_counts=True)
    for val, cnt in zip(unique, counts):
        print(f"  {int(val)} 年: {cnt:,} 個像素")


# 建立一個帶有正確統計資訊的 GeoTIFF
import rasterio
import numpy as np
from pathlib import Path

data_dir = Path("/Users/insight_ai/我的雲端硬碟/share/防災materials/nasa/VNP46A4_1-20251004_081026")

# 重新儲存，確保統計資訊正確
input_file = data_dir / "VIIRS_multi_year_mean.tif"
output_file = data_dir / "VIIRS_multi_year_mean_v2.tif"

with rasterio.open(input_file) as src:
    data = src.read(1)
    profile = src.profile.copy()
    
    # 計算統計資訊
    valid_data = data[~np.isnan(data)]
    stats = {
        'min': float(valid_data.min()),
        'max': float(valid_data.max()),
        'mean': float(valid_data.mean()),
        'std': float(valid_data.std())
    }
    
    print(f"重新儲存檔案，統計資訊:")
    for key, value in stats.items():
        print(f"  {key}: {value:.6f}")
    
    # 寫入新檔案
    with rasterio.open(output_file, 'w', **profile) as dst:
        dst.write(data, 1)
        
        # 寫入統計資訊到元資料
        dst.update_tags(
            1,
            STATISTICS_MINIMUM=str(stats['min']),
            STATISTICS_MAXIMUM=str(stats['max']),
            STATISTICS_MEAN=str(stats['mean']),
            STATISTICS_STDDEV=str(stats['std'])
        )
        
        dst.update_tags(
            description='Multi-year mean of 4 years (2020-2023)',
            units='nW/cm2/sr',
            statistics='mean',
            valid_range=f"{stats['min']:.2f} to {stats['max']:.2f}"
        )

print(f"\n✓ 已重新儲存至: {output_file.name}")
print("請在 QGIS 中開啟此新檔案並檢查數值範圍")

=== 驗證 GeoTIFF 檔案 ===

檔案: VIIRS_multi_year_mean.tif
資料類型: float32
形狀: (2400, 2400)
CRS: EPSG:4326
NoData: nan

數值統計:
  最小值: 0.000000
  最大值: 259.274994
  平均值: 4.359822
  中位數: 0.675000
  標準差: 10.305140

百分位數:
  25%: 0.000000
  50%: 0.675000
  75%: 3.425000
  95%: 22.674999
  99%: 52.924999

NaN 值統計:
  總像素數: 5760000
  NaN 數量: 5277692
  有效數據: 482308

元資料:
  description: Multi-year mean of 4 years
  source_files: VNP46A4.A2023001.h30v06.001.2024019015120.h5, VNP46A4.A2021001.h30v06.001.2022094124511.h5, VNP46A4.A2022001.h30v06.001.2023082112411.h5, VNP46A4.A2020001.h30v06.001.2021125121127.h5
  statistics: mean
  units: nW/cm2/sr
  AREA_OR_POINT: Area

有效資料範圍:
  最小值: 0.000000
  最大值: 259.274994

=== 檢查標準差檔案 ===

標準差範圍: 0.000000 到 265.731995

=== 檢查有效年份數檔案 ===

有效年份數:
  0 年: 5,277,692 個像素
  1 年: 2,562 個像素
  2 年: 1,432 個像素
  3 年: 1,311 個像素
  4 年: 477,003 個像素
重新儲存檔案，統計資訊:
  min: 0.000000
  max: 259.274994
  mean: 4.359820
  std: 10.305142

✓ 已重新儲存至: VIIRS_multi_year_mean_v2.tif
請在 QGIS 中開啟此新檔案